In [71]:
import PyPDF2
from tqdm import tqdm
import regex as re
import pandas as pd
from collections import Counter
import spacy

In [ ]:
# Powitanie i zakończenia listów.
# Czasowniki: prosić, informować, zachęcać.
# Wyrażenia: z przyjemnością, z przykrością.
# Zwroty: Szanowni Państwo, Szanowna Pani.
# Ilości listów do mnie vesus ilość listów ode mnie do szkoły.

In [60]:
with open('korpus wiadomosci szkola -- sample.pdf', 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    text = []
    for page in reader.pages:
        text.append(page.extract_text())

In [61]:
merged_text = []
for elem in text:
    if elem.startswith('Skrzynka:'):
        merged_text.append(elem)
    else:
        merged_text[-1] = merged_text[-1] + '\n' + elem
del text

In [62]:
letters_dicts = []
id_counter = 1
for elem in merged_text:
    elem_dict = {'id': id_counter}
    elem_dict['Skrzynka'] = re.search('(?<=Skrzynka:).+', elem).group(0).strip()
    elem_dict['Nadawca'] = re.search('(?<=Nadawca:).+', elem).group(0).strip()
    elem_dict['Otrzymano'] = re.search('(?<=Otrzymano:).+', elem).group(0).strip()
    elem_dict['Odczytano'] = re.search('(?<=Odczytano:).+', elem).group(0).strip()
    elem_dict['Temat'] = re.search('(?<=Temat:).+', elem).group(0).strip()
    elem_dict['Tekst'] = '\n'.join(elem.splitlines()[5:])
    letters_dicts.append(elem_dict)
    id_counter += 1
    if 'Od: Jurewicz Magdalena' in elem_dict['Tekst']:
        subletters = elem_dict['Tekst'].split('Od: ')
        elem_dict['Tekst'] = subletters[0]
        subletters = subletters[1:]
        subletters = ['Od: ' + e for e in subletters]
        sub_id_counter = 1
        for sub in subletters:
            subletter_dict = {'id': f"{elem_dict['id']}_{sub_id_counter}"}
            subletter_dict['Skrzynka'] = elem_dict['Nadawca']
            subletter_dict['Nadawca'] = re.search('(?<=Od:).+', sub).group(0).strip()
            subletter_dict['Otrzymano'] = re.search('(?<=Wysłano:).+', sub).group(0).strip()
            subletter_dict['Odczytano'] = ''
            subletter_dict['Temat'] = re.search('(?<=Temat:).+', sub).group(0).strip()
            subletter_dict['Tekst'] = '\n'.join(sub.splitlines()[3:])
            letters_dicts.append(subletter_dict)
            sub_id_counter += 1

new_letters_dicts = []
subjects = set()
for letter in letters_dicts:
    if (letter['Temat'], letter['Otrzymano']) not in subjects:
        new_letters_dicts.append(letter)
        subjects.add((letter['Temat'], letter['Otrzymano']))

letters_dicts = new_letters_dicts

for letter in letters_dicts:
    txt_to_save = f'''Odbiorca: {letter['Skrzynka']}
Nadawca: {letter['Nadawca']}
Otrzymano: {letter['Otrzymano']}
Odczytano: {letter['Odczytano']}
Temat: {letter['Temat']}
Tekst:

{letter['Tekst']}
    '''
    with open(f'letters_sample/{letter["id"]}.txt', 'w', encoding='utf-8') as file:
        file.writelines(txt_to_save)


In [ ]:
# Powitanie i zakończenia listów.
greetings = []
for elem in letters_dicts:
    if elem['Tekst']:
        greet = elem['Tekst'].splitlines()[0]
        if len(greet) <= 55:
            greetings.append(greet.strip(' .,'))
print(Counter(greetings))

goodbyes = []
for elem in letters_dicts:
    if elem['Tekst']:
        # delete links at the end of text
        txt = re.sub(r'https?://\S+', '', elem['Tekst']).strip()
        bye = txt.splitlines()[-2:]
        bye = '\n'.join(bye)
        goodbyes.append(bye.strip())
print(Counter(goodbyes))

greetings_dct = {}
for greet, _ in Counter(greetings).items():
    ids = []
    for elem in letters_dicts:
        if elem['Tekst'] and elem['Tekst'].splitlines()[0].strip(' .,') == greet:
            ids.append(elem['id'])
    greetings_dct[greet] = {
        'ids': ids,
        'num': len(ids)
    }
print(greetings_dct)

goodbyes_dct = {}
for bye, _ in Counter(goodbyes).items():
    ids = []
    for elem in letters_dicts:
        if bye in elem['Tekst']:
            ids.append(elem['id'])
    goodbyes_dct[bye] ={
        'ids': ids,
        'num': len(ids)
    }
print(goodbyes_dct)

In [86]:
df = pd.DataFrame.from_dict(greetings_dct, orient="index").sort_values(by="num", ascending=False)
df.to_excel('powitania.xlsx', encoding='utf-8')

df = pd.DataFrame.from_dict(goodbyes_dct, orient="index").sort_values(by="num", ascending=False)
df.to_excel('pozegnania.xlsx', encoding='utf-8')

In [ ]:
# Czasowniki: prosić, informować, zachęcać.
nlp = spacy.load("pl_core_news_lg")

czasowniki_prosic = []
czasowniki_informowac = []
czasowniki_zachecac = []
for letter in letters_dicts:
    doc = nlp(letter['Tekst'].lower())

    bezokolicznik = 'prosić'
    czasowniki = [token.text for token in doc if token.pos_ == "VERB" and token.lemma_ == bezokolicznik]
    czasowniki_prosic.extend(czasowniki)

    bezokolicznik = 'informować'
    czasowniki = [token.text for token in doc if token.pos_ == "VERB" and token.lemma_ == bezokolicznik]
    czasowniki_informowac.extend(czasowniki)

    bezokolicznik = 'zachęcać'
    czasowniki = [token.text for token in doc if token.pos_ == "VERB" and token.lemma_ == bezokolicznik]
    czasowniki_zachecac.extend(czasowniki)

print(Counter(czasowniki_prosic))
print(Counter(czasowniki_informowac))
print(Counter(czasowniki_zachecac))

In [ ]:
# Wyrażenia: z przyjemnością, z przykrością.
counter = 0
ids = []
for elem in letters_dicts:
    num = elem['Tekst'].lower().count('z przyjemnością')
    if num > 0:
        counter += num
        ids.append(elem['id'])
print(counter, ids)

counter = 0
ids = []
for elem in letters_dicts:
    num = elem['Tekst'].lower().count('z przykrością')
    if num > 0:
        counter += num
        ids.append(elem['id'])
print(counter, ids)

In [ ]:
# Zwroty: Szanowni Państwo, Szanowna Pani.
counter = 0
ids = []
for elem in letters_dicts:
    num = elem['Tekst'].lower().count('szanowni państwo')
    if num > 0:
        counter += num
        ids.append(elem['id']) 
print(counter, ids)

counter = 0
ids = []
for elem in letters_dicts:
    num = elem['Tekst'].lower().count('szanowna pani')
    if num > 0:
        counter += num
        ids.append(elem['id'])
print(counter, ids)

In [ ]:
# Ilości listów do mnie vesus ilość listów ode mnie do szkoły.
counter1 = 0
counter2 = 0
for elem in letters_dicts:
    if elem['Nadawca'].startswith('Jurewicz Magdalena'):
        counter1 += 1
    else: counter2 += 1
print(counter1, counter2)